In [2]:
import pandas as pd
import ast
from collections import Counter

nrc = pd.read_csv("NRC-Emotion-Lexicon-Wordlevel-v0.92.txt", 
                  sep='\t', names=["word", "emotion", "association"])
nrc = nrc[nrc['association'] == 1].drop(columns="association")

emotion_dict = {}
for word, emotion in nrc.values:
    emotion_dict.setdefault(word, []).append(emotion)

emotion_to_emoji = {
    'joy': '😊',
    'sadness': '😢',
    'anger': '😠',
    'fear': '😨',
    'surprise': '😲',
    'disgust': '🤢',
    'trust': '🤝',
    'anticipation': '🤔'
}

df = pd.read_csv("predictions_ordered.csv").iloc[:10]

for _, row in df.iterrows():
    text = row['text']
    try:
        keywords = ast.literal_eval(row['kws'])
    except:
        keywords = []

    emotion_counter = Counter()
    
    for kw in keywords:
        kw_lower = kw.lower()
        if kw_lower in emotion_dict:
            for emotion in emotion_dict[kw_lower]:
                emotion_counter[emotion] += 1

    if emotion_counter:
        top_emotion = emotion_counter.most_common(1)[0][0]
        emoji = emotion_to_emoji.get(top_emotion, '❓')
    else:
        emoji = '😐'
        
    print(f"{emoji} {text}")


😲 gt The decoy device was given to him by an undercover FBI agent, the statement said. As usual.
❓ gt 4profit!!! You keep using that word, but I don't think it means what you think it means...
❓ U folks from my congressional district rock! LOL!! kthxbai
😐 No, you have ABS on the front brakes, which are discs. Those drums will happily lock up and oversteer. EDIT: I have no idea what I'm talking about.
❓ []Nidies 62 points 2 hours ago You're a jackass. Sincerely, All the people that actually need a GM and have had to wait for days to talk to one before. That's a repost too... weird..
🤝 I get mine inland marine insurance from state farm.
😠 The mob textures seem to be broken. All my enderman and black spiders are white...
😠 You get an up vote for making me feel like I've accomplished something other than playing Bad Company 2 today.
🤔 I never said anything about the hand of God. You're the one that jumped to that conclusion.
🤝 YOU MUST USE YOUR SUBREDDIT AS RLIBERTARIAN DEMANDS. ONLY BY DO